# Animal Attacks in North America

In [330]:
#import packages
from lxml import html
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [331]:
#function to get rows in a table
def get_table_rows(table):
    l = []
    table_rows = table.find_all('tr')
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        l.append(row)
    return l

In [332]:
#function to get rows in table after marker
def get_rows_table(tables):
    tb=[]
    for barbie in tables:
        tables=barbie.findNext('table')
        tb.append(tables)
    l = []

    for table in tb:
        if table is not None:
            table_rows = table.find_all('tr')
            row = []
            for tr in table_rows:
                td = tr.find_all('td')
                row = [tr.text for tr in td]
                l.append(row)
    return l

In [333]:
def get_rows_all_tables(tables):
    l=[]
    for tb in tables:
        table_rows = tb.find_all('tr')
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
    return l

In [334]:
# get all locations hrefs in the n-th column
def extract_href_location(tables,n):
    l=[]
    for tb in tables:
        table_rows = tb.find_all('tr')
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            for tt in td:
                ref=tt.find('a', href=re.compile(r'.*wiki*'))
                if ref:
                    title = ref.get('title')
                    l.append(title)
                else:
                    l.append('NaN')
    return l[n-1::n]

In [335]:
#install packages
#!pip install geocoder
#!pip install folium==0.5.0
#!conda install -c conda-forge folium=0.5.0 --yes

In [336]:
#import packages to draw on map
import geocoder
import folium
from geopy.geocoders import Nominatim

In [337]:
# create map of North America using latitude and longitude values
address = 'Vancouver, BC'

geolocator = Nominatim(user_agent="na_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_north_america = folium.Map(location=[latitude, longitude], zoom_start=3)
map_north_america_pure = map_north_america

In [338]:
def add_coordinates(table):
    lat_lng_coords = None
    latlong=[]
    for n in range(0,len(table)):
        g = geocoder.arcgis(table['Location'].values[n])
        lat_lng_coords = g.latlng
        latlong.append(lat_lng_coords)
    laa = np.array(latlong)
    latitude=laa[:,0].tolist()
    longitude=laa[:,1].tolist()
    lat=pd.DataFrame(latitude,columns=['Latitude']) 
    long=pd.DataFrame(longitude,columns=['Longitude']) 
    lulo=pd.concat([lat,long], axis=1)
    data=pd.concat([table,lulo],axis=1)
    return data

In [339]:
def add_to_map(table,colour,fill_colour,map_of):
    for lat, lng, Loc in zip(table['Latitude'], table['Longitude'], table['Location']):
        label = '{}'.format(Loc)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker([lat, lng], radius=3, popup=label, color=colour, fill=True, fill_color=fill_colour, fill_opacity=0.7, parse_html=False).add_to(map_of)
    return map_of

##  Bear Attacks in North America

Scrape and Explore Dataset

In [340]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_fatal_bear_attacks_in_North_America")
soup = BeautifulSoup(r.content, "html.parser")

In [341]:
dflist=[]
for bear_type in ("Polar","Black","Brown"):
    bears=soup.findAll(title="Edit section: "+bear_type+" bear")
    bear=get_rows_table(bears)
    df=pd.DataFrame(bear, columns=["Person", "Date", "Type", "Location", "Details"])
    df['Bear']=bear_type
    dflist.append(df)
all_attacks=pd.concat(dflist).replace(r'\n',' ', regex=True).dropna().reset_index(drop=True)

In [342]:
data=add_coordinates(all_attacks)
data=data.loc[data['Type']=='Wild'].reset_index(drop=True)
pb=data.loc[data['Bear']=='Polar']
blb=data.loc[data['Bear']=='Black']
brb=data.loc[data['Bear']=='Brown']

In [343]:
map_bear_attacks=add_to_map(brb,'brown','#3186cc',add_to_map(blb,'black','#3186cc',add_to_map(pb,'blue','#3186cc',map_north_america)))
map_bear_attacks;

# Wolf Attacks

In [344]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_wolf_attacks_in_North_America")
soup = BeautifulSoup(r.content, "html.parser")

In [345]:
fatals=soup.find('table')
df=pd.DataFrame(get_table_rows(fatals), columns=["Victim(s)","Age","Sex","Date","Type of attack","Location","Details","Source(s)"])
ftl_wolf_attacks = df.replace(r'\n',' ', regex=True).dropna().reset_index(drop=True)

In [346]:
wvs=add_coordinates(ftl_wolf_attacks)

In [347]:
#add_to_map(wvs,'gray','gray',map_north_america)
add_to_map(wvs,'gray','gray',map_north_america_pure);

# Cougar Attacks

In [348]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_fatal_cougar_attacks_in_North_America")
soup = BeautifulSoup(r.content, "html.parser")

In [349]:
cougars=soup.findAll('table',{"class":"wikitable sortable"})
#puma=[x for x in get_table_rows(cougars) if x != []]
puma=[x for x in get_rows_all_tables(cougars) if x != []]
cgr_attacks=pd.DataFrame(puma, columns=["Name, Age, Gender", "Date","Location"]).replace(r'\n',' ', regex=True).dropna().reset_index(drop=True)
#cgr_attacks=df.replace(r'\n',' ', regex=True).dropna().reset_index(drop=True)

In [350]:
# Extract Age
# cgr_attacks['Name, Age, Gender'].str.extract(r'([\d]+)',expand=False)

In [351]:
# Overwrite Location with href-Location-data
cgr_attacks["Location"]=pd.DataFrame(extract_href_location(cougars,3))

In [352]:
cuga=add_coordinates(cgr_attacks)

In [353]:
add_to_map(cuga,'yellow','gray',map_north_america)

# Snake bites in the US

In [354]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_fatal_snake_bites_in_the_United_States")
soup = BeautifulSoup(r.content, "html.parser")

# Clean Map
## Erase coordinates outside of North America